In [1]:
#import packages
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import scipy
import requests
from imdbpie import Imdb
import nltk
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
import nltk
import collections
import re
import csv
import psycopg2
from scrapy.selector import Selector
import os
import spacy
from spacy.en import English
parser = English()
import unicodedata
import random
from time import sleep
import time



%matplotlib inline

In [2]:
# connect to the IMDB api
imdb = Imdb()
imdb = Imdb(anonymize=True)

In [3]:
#get the top 250 movies
top = imdb.top_250()
#put the top 25 movies in a dataframe
top250 = pd.DataFrame(top)
#drop useless columns
top250 = top250.drop('image', axis=1).drop('can_rate',axis=1)
#export to csv
top250.to_csv('top250.csv',encoding='utf-8')
#store the codes of the top 25 movies in the variable tconst
titleList = [i for i in top250.tconst]

In [ ]:
# reviewList2 = []

# for i in titleList[0:2]:
#     title5 = imdb.get_title_by_id(i)
#     user2 = imdb.get_title_reviews(i, max_results = 5)
#     for user in user2:
#         reviewList2.append((title5.title, user.text, user.rating, title5.rating))

In [4]:
titleList= titleList[:25]

In [ ]:
# pd.DataFrame(reviewList2, columns = ['title','reviews','review_score','movie_score'])

In [ ]:
# actioncomedy = ['tt1343103','tt2574698','tt5720450','tt0338459','tt4335650','tt2381991','tt2975590','tt1431045','tt0468569','tt0252487','tt4048272','tt2278388','tt0163651','tt3416828','tt2637294','tt0108255','tt1107812','tt0362165','tt4404474']

In [ ]:
# len(actioncomedy)

In [5]:
#titlelist2 is a list of movies from 6 genres. Each genre has a range scores from 1 to 10
titleList2 = ['tt1343103','tt2574698','tt5720450','tt0338459','tt4335650','tt2381991','tt2975590','tt1431045','tt0468569','tt0252487','tt4048272','tt2278388','tt0163651','tt3416828','tt2637294','tt0108255','tt1107812','tt0362165','tt4404474','tt0107715',
 'tt0185183',
 'tt0115624',
 'tt0358349',
 'tt0114614',
 'tt0097428',
 'tt0120903',
 'tt0133093',
 'tt5466576','tt2313780',
 'tt0367677',
 'tt0093300',
 'tt3832096',
 'tt1131734',
 'tt4094724',
 'tt0084516',
 'tt0081505',
 'tt4687276','tt0110357', 'tt0126029', 'tt0120855', 'tt0371606', 'tt2017020', 'tt1594972', 'tt0763304', 'tt0330994','tt1623780', 'tt1702443', 'tt1229827', 'tt3797808', 'tt0939684', 'tt1753433', 'tt4382552', 'tt3962848', 'tt5278596',
         'tt2653342', 'tt1754151']

In [6]:
titleList3 = titleList2 + titleList

In [7]:
len(titleList3)

81

In [ ]:
# reviewList4 = []

# for i in titleList3:
#     title5 = imdb.get_title_by_id(i)
#     user2 = imdb.get_title_reviews(i, max_results = 200)
#     for user in user2:
#         reviewList4.append((title5.title, user.text, user.rating, title5.rating, title5.genres[0]))

In [ ]:
# df = pd.DataFrame(reviewList3,columns = ['title','reviews','review_score','movie_score','genre'])

In [ ]:
# df.shape

In [ ]:
#are there null values in the DF?
# df.isnull().sum()

In [8]:
#make a list of reviews with movie title, user review, review score, movie score, and genre
reviewList5 = []

for i in titleList3:
    title5 = imdb.get_title_by_id(i)
    time.sleep(random.random())
    user2 = imdb.get_title_reviews(i, max_results = 200)
    for user in user2:
        reviewList5.append((title5.title, user.text, user.rating, title5.rating, title5.genres[0]))

In [10]:
#turn the list of reviews into a dataframe
df = pd.DataFrame(reviewList5, columns = ['title','reviews','review_score','movie_score','genre'])

In [ ]:
# df.tail()

In [ ]:
# np.mean(df.review_score)

In [ ]:
# type(df.review_score[3])

In [11]:
#fill na in review score with movie score
df.review_score.fillna(df.movie_score, inplace =True)

In [12]:
df.isnull().sum()

title           0
reviews         0
review_score    0
movie_score     0
genre           0
dtype: int64

In [ ]:
# export the df to csv
# df.to_csv('ALLMovieData200.csv', encoding= 'utf-8')

In [ ]:
# list1 = df['reviews'][0]

In [ ]:
# list1

In [ ]:
# parsed = parser(unicode(df['reviews'][0]))

In [ ]:
# parsed.text

In [ ]:
# lemmas_unicode = [token.lemma_ for token in parsed]

In [ ]:
# lemmas_string = [unicodedata.normalize('NFKD', i).encode('utf-8') for i in lemmas_unicode]

In [ ]:
# lstring = ''
# for i in lemmas_string:
#     if i not in  ['\n\n', '!','?','.',',','[0-9]']:
#         lstring += i
#         lstring += ' '
# lstring

In [13]:
def cleaner(string):
    parsed = parser(unicode(string))
    lemmas_unicode = [token.lemma_ for token in parsed]
    lemmas_string = [unicodedata.normalize('NFKD', i).encode('utf-8') for i in lemmas_unicode]
    lstring = ''
    for i in lemmas_string:
        if i not in  ['\n\n', '!','?','.',',','[0-9]+','[)()-\'/]']:
            lstring += i
            lstring += ' '
    return lstring

In [ ]:
# cleaner(df.reviews[1])

In [14]:
reviewlist = []
for i in df['reviews']:
    reviewlist.append(cleaner(i))
reviewlist[:10]

['i love movie and i also will check out most independent produce picture as well when i saw this have a small but functional budget i give it a chance all i can say be this the best act in this movie come from the animated creature the script be weak and they manage to include almost every stereotype that have ever exist yet not one of them could act seriously you will spend time that you can never get back and life be just too short to care do not waste your time just my thought ',
 'even from the trailer raptor ranch from the get go be one such movie to not expect too much from and to not take too seriously the trailer do look sort of fun if in a guilty - pleasure sort of way and any movie deserve to be see fairly sad to say that from personal opinion raptor ranch be every bit as bad as the review and rating here say(and no i be not one of those immature people who vote 1 literally a year before raptor ranch be even release ) one of those instance where the ridiculously hyperbolic i

In [15]:
df['reviews'] = reviewlist

In [16]:
df.head()

,title,reviews,review_score,movie_score,genre
0,Raptor Ranch,i love movie and i also will check out most in...,2.0,3.0,Action
1,Raptor Ranch,even from the trailer raptor ranch from the ge...,1.0,3.0,Action
2,Raptor Ranch,well this movie have prehistoric raptor and lo...,2.0,3.0,Action
3,Raptor Ranch,' raptor ranch ' ( audaciously retitled ' jura...,3.0,3.0,Action
4,Raptor Ranch,personally i do not get all the hate for this ...,7.0,3.0,Action


In [17]:
#export the df to csv
df.to_csv('ALLMovieData200.csv')